In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName('aws') \
        .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1646528662074_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data retrieval

## Get data from s3

In [2]:
from urllib.request import urlopen

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
lines_2014 = sc.textFile('s3://gdelt-open-data/events/2014*')
parts_2014 = lines_2014.map(lambda l: l.split('\t'))

lines_2015 = sc.textFile('s3://gdelt-open-data/events/2015*')
parts_2015 = lines_2015.map(lambda l: l.split('\t'))

lines_2016 = sc.textFile('s3://gdelt-open-data/events/2016*')
parts_2016 = lines_2016.map(lambda l: l.split('\t'))

lines_2017 = sc.textFile('s3://gdelt-open-data/events/2017*')
parts_2017 = lines_2017.map(lambda l: l.split('\t'))

lines_2018 = sc.textFile('s3://gdelt-open-data/events/2018*')
parts_2018 = lines_2018.map(lambda l: l.split('\t'))

lines_2019 = sc.textFile('s3://gdelt-open-data/events/2019*')
parts_2019 = lines_2019.map(lambda l: l.split('\t'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Convert RDD into DataFrame

In [4]:
html = urlopen('http://gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt').read().rstrip().decode('utf8')
columns = html.split('\t')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from functools import reduce
from pyspark.sql import DataFrame

def union_all(*dfs):
    return reduce(DataFrame.unionByName, dfs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_2014 = spark.createDataFrame(parts_2014, columns)
df_2015 = spark.createDataFrame(parts_2015, columns)
df_2016 = spark.createDataFrame(parts_2016, columns)
df_2017 = spark.createDataFrame(parts_2017, columns)
df_2018 = spark.createDataFrame(parts_2018, columns)
df_2019 = spark.createDataFrame(parts_2019, columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Join 2014 and 2019 dataframes

In [7]:
my_dic = {}
my_dic['df_2014'] = df_2014
my_dic['df_2015'] = df_2015
my_dic['df_2016'] = df_2016
my_dic['df_2017'] = df_2017
my_dic['df_2018'] = df_2018
my_dic['df_2019'] = df_2019

df = union_all(*my_dic.values())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Remove rows tagged as <= 2013

In [8]:
df = df.filter((df.Year >= 2014))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Save as parquet

In [ ]:
df.write.parquet('s3://bdcc-lab2-bucket/newer_df.parquet', partitionBy=['Year', 'MonthYear'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check file

## Print schema

In [8]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: stri

# View first 5 items

In [17]:
import pandas as pd
df.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  GLOBALEVENTID  ...                                          SOURCEURL
0     813415774  ...  https://www.weyburnreview.com/cn-rail-and-cp-r...
1     813415775  ...  http://www.startribune.com/arcelormittal-takes...
2     813415776  ...  https://www.weyburnreview.com/cn-rail-and-cp-r...
3     813415777  ...  https://www.weyburnreview.com/cn-rail-and-cp-r...
4     813415778  ...  https://globalnews.ca/news/4805430/professor-p...

[5 rows x 58 columns]

# Number of rows

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

356198797

In [8]:
from pyspark.sql.functions import desc

(newer_df.groupby('year')
   .count()
   .orderBy(desc('year'))
   .show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|year|   count|
+----+--------+
|2019|40763553|
|2018|61231746|
|2017|66269395|
|2016|73269882|
|2015|66282411|
|2014|48381810|
+----+--------+